# Create a Pharos class input file

This notebook creates a file that contains gene symbols, Ensembl IDs, UniProt IDs, and Pharos class for each gene. The steps are:
1. Query Pharos for all targets in their database
2. Parse the JSON response as a data frame
3. Merge this data with Agora's existing UniProt -> Ensembl ID map file

Step 3 simultaneously maps the IDs given by Pharos (UniProt IDs and gene symbols only) to the corresponding Ensembl IDs, and narrows the data down to only genes that exist in Agora's data files.

In [1]:
import requests
import json
import pandas as pd
import synapseclient

## Query Pharos for all targets in the database

Pharos uses GraphQL for its API, so the query is formatted as JSON that matches their "DownloadResult" schema. The lone "data" value inside signifies to return the data only, not any status or metadata about the request.

In [2]:
url = "https://pharos-api.ncats.io/graphql"

query_body = """
{
    download(model: "Targets", fields: ["UniProt", "Symbol", "Target Development Level"], sqlOnly: false) {
        data
    }
}
"""

response = requests.post(url=url, json={"query": query_body})

if not response.ok:
    print("Error querying Pharos")
    response.raise_for_status()

## Turn the response into a DataFrame

The response is a JSON string with the following structure:

```
data {
    download {
        data {
            [list of dictionary items with fields "id", "UniProt", "Symbol", and "Target Development Level"]]
        }
    }
}
```

Calling `json_normalize` on the inner "data" item will turn that list into a data frame. 

In [3]:
res_str = json.loads(response.content)
pharos_df = pd.json_normalize(res_str["data"]["download"]["data"])

pharos_df = pharos_df.rename(
    columns={
        "UniProt": "uniprot_id",
        "Symbol": "hgnc_symbol",
        "Target Development Level": "pharos_class",
    }
).drop(columns="id")

pharos_df

,uniprot_id,hgnc_symbol,pharos_class
0,P32929,CTH,Tchem
1,A4D0Y5,C7orf77,Tdark
2,Q49A92,C8orf34,Tbio
3,Q9UFW8,CGGBP1,Tbio
4,Q96K31,C8orf76,Tdark
...,...,...,...
20407,P49286,MTNR1B,Tclin
20408,P84157,MXRA7,Tdark
20409,Q9NP71,MLXIPL,Tbio
20410,Q9BWT6,MND1,Tbio


## Map UniProt IDs to Ensembl IDs

Uses the UniProt -> Ensembl ID Agora source file

In [4]:
# Temporarily hard-coded until this file is in the config.yaml file
uniprot_syn_id = "syn54113663"

syn = synapseclient.Synapse()
syn.login(silent=True)

uniprot_path = syn.get(uniprot_syn_id)
uniprot_df = pd.read_table(uniprot_path.path, sep="\t")

uniprot_df


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.6.0) is available. Your version (4.0.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.6.0 release notes

https://python-docs.synapse.org/news/



,UniProtKB_accession,RESOURCE_IDENTIFIER
0,A0A075B6I4,ENSG00000211642
1,Q13641,ENSG00000146242
2,Q6PCB7,ENSG00000130304
3,Q7Z591,ENSG00000106948
4,Q5SZD1,ENSG00000197261
...,...,...
18456,Q6ZUI0,ENSG00000188001
18457,O43747,ENSG00000166747
18458,Q9UBU2,ENSG00000155011
18459,Q86VY9,ENSG00000164484


In [5]:
final_df = pd.merge(
    left=pharos_df,
    right=uniprot_df,
    how="inner",
    left_on="uniprot_id",
    right_on="UniProtKB_accession",
).rename(columns={"RESOURCE_IDENTIFIER": "ensembl_gene_id"})

final_df = final_df[
    ["ensembl_gene_id", "uniprot_id", "hgnc_symbol", "pharos_class"]
].sort_values(by="ensembl_gene_id")

final_df

,ensembl_gene_id,uniprot_id,hgnc_symbol,pharos_class
8825,ENSG00000000003,O43657,TSPAN6,Tbio
4568,ENSG00000000005,Q9H2S6,TNMD,Tbio
194,ENSG00000000419,O60762,DPM1,Tbio
3186,ENSG00000000457,Q8IZE3,SCYL3,Tbio
11997,ENSG00000000460,Q9NSG2,C1orf112,Tbio
...,...,...,...,...
7879,ENSG00000288705,P35504,UGT1A5,Tbio
1485,ENSG00000288722,P23610,F8A1,Tbio
1682,ENSG00000288784,Q8IX94,CTAGE4,Tdark
2959,ENSG00000289721,W6CW81,PYDC5,Tbio


## Save to a file

In [6]:
final_df.to_csv("../output/pharos_classes.csv", index=False)